# Clustering

### Hierarchical clustering:
- Clustering approach in which cluster boundaries are nested to each other, creating a parent/child relationship.
- This nested structure between clusters creates a hierarchy, that is often represented in the form of a cluster tree, known as dendogram

### Partitional Clustering:
- Each cluster boundary is independent of the others. There is no hierarchy between clusters. Dividing data objects into non overlapping groups.
- No object can be a member of more than one cluster, and each cluster have at least one object

### Overlaping Clustering
- Approach in which the boundary of one cluster can overlap that of another. Each object of the dataset can be member of one or more clusters. 
- There is no parent-child relationship here, the boundaries of the child not always are in the boundaries of the parent

### Fuzzy/soft clustering
- the membership of an item to a particulary cluster, is based on a membership weight that goes between 0 and 1. The larger the weight, the greater the item belongs to a particular cluster.
- If the weight is 0, the item absolutely does not belong to the cluster. If the weight is 1, then the item absolutely belong to the cluster.

### Density based clustering
- Determine cluster assignments based on the density of data points in a region
- Clusters are assign where there are high densities of data points separated by low density regions

## The k-Means belongs to partitional clustering
- How many clusters 'k' do we want?
- The algorithm uses a process known as expectation maximization to assign every item to one and only one cluster
- Calculates the Euclidian distance between each point and each of the clusters centers

### How to choose the right 'k'?
- Based on a priori knowledge or business requirements
- Using a rule of thumb, such as k ~= (n/2)^(1/2). But only with small datasets
- Using one or more statistical measures, such as the Elbow method, the average Silhouete method and the Calinski-Harabasz Score